In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
from tqdm import tqdm

import warnings, os
import mysql.connector as mysql
warnings.filterwarnings('ignore')
username = os.environ['MYSQL_user']
password = os.environ['MYSQL_password']
DB = mysql.connect(host = "localhost", user = username, passwd = password, database = "AIRBNB")
cursor = DB.cursor(buffered=True)

In [2]:
def read_table_from_db(table_name):
    df = pd.read_sql(f'SELECT * FROM {table_name}', con=DB)
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    df = gpd.GeoDataFrame(df, geometry='geometry')
    return df

# Crimes

In [3]:
crimes_data = pd.read_csv('Data/NYPD_crimes.csv')
crimes_data.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,287066200,123,STATEN ISLAND,05/17/2024,22:21:00,05/17/2024,22:25:00,COMPLETED,(null),NaN,...,NaN,45-64,WHITE,F,937116.0,137959.0,40.545217,-74.169591,"(40.545217, -74.169591)",POINT (-74.169591 40.545217)
1,287424426,121,STATEN ISLAND,05/24/2024,17:20:00,05/24/2024,17:25:00,COMPLETED,(null),NaN,...,NaN,25-44,WHITE,F,937288.0,151471.0,40.582305,-74.169067,"(40.582305, -74.169067)",POINT (-74.169067 40.582305)
2,290583574,121,STATEN ISLAND,06/05/2024,07:00:00,06/10/2024,07:00:00,COMPLETED,(null),NaN,...,NaN,65+,BLACK,F,937791.0,172099.0,40.638927,-74.167396,"(40.638927, -74.167396)",POINT (-74.167396 40.638927)
3,289989348,121,STATEN ISLAND,07/12/2024,18:50:00,07/12/2024,19:05:00,COMPLETED,(null),NaN,...,NaN,UNKNOWN,UNKNOWN,E,938881.0,167697.0,40.626851,-74.163438,"(40.626851, -74.163438)",POINT (-74.163438 40.626851)
4,283856947,121,STATEN ISLAND,03/16/2024,19:45:00,03/16/2024,20:00:00,COMPLETED,(null),NaN,...,NaN,25-44,UNKNOWN,M,940986.0,169325.0,40.631331,-74.155867,"(40.631331, -74.155867)",POINT (-74.155867 40.631331)


In [4]:
crimes_unique = crimes_data['OFNS_DESC'].unique().tolist()
print(crimes_unique)

['HARRASSMENT 2', 'FRAUDS', 'DANGEROUS WEAPONS', 'FELONY ASSAULT', 'ARSON', 'ROBBERY', 'BURGLARY', 'SEX CRIMES', 'MURDER & NON-NEGL. MANSLAUGHTER', 'ASSAULT 3 & RELATED OFFENSES', 'GRAND LARCENY', 'CRIMINAL MISCHIEF & RELATED OF', 'OFFENSES INVOLVING FRAUD', 'PETIT LARCENY', 'VEHICLE AND TRAFFIC LAWS', 'OFF. AGNST PUB ORD SENSBLTY &', 'INTOXICATED & IMPAIRED DRIVING', 'OFFENSES AGAINST PUBLIC ADMINI', 'CRIMINAL TRESPASS', 'OTHER STATE LAWS', 'OFFENSES AGAINST THE PERSON', 'MISCELLANEOUS PENAL LAW', 'GRAND LARCENY OF MOTOR VEHICLE', 'FORGERY', 'UNAUTHORIZED USE OF A VEHICLE', 'DANGEROUS DRUGS', 'OTHER STATE LAWS (NON PENAL LAW)', 'THEFT-FRAUD', 'DISORDERLY CONDUCT', 'ADMINISTRATIVE CODE', 'RAPE', 'KIDNAPPING & RELATED OFFENSES', 'PROSTITUTION & RELATED OFFENSES', 'POSSESSION OF STOLEN PROPERTY', 'CHILD ABANDONMENT/NON SUPPORT 1', 'PETIT LARCENY OF MOTOR VEHICLE', 'DISRUPTION OF A RELIGIOUS SERV', 'CANNABIS RELATED OFFENSES', 'OTHER OFFENSES RELATED TO THEFT', 'JOSTLING', "BURGLAR'S TOOL

I will map crimes with categories from this site: https://www150.statcan.gc.ca/n1/pub/85-004-x/2009001/t001-eng.htm

In [5]:
crimes = ['HARRASSMENT 2', 'FRAUDS', 'DANGEROUS WEAPONS', 'FELONY ASSAULT', 'ARSON', 'ROBBERY', 'BURGLARY', 'SEX CRIMES', 'MURDER & NON-NEGL. MANSLAUGHTER', 'ASSAULT 3 & RELATED OFFENSES', 'GRAND LARCENY', 'CRIMINAL MISCHIEF & RELATED OF', 'OFFENSES INVOLVING FRAUD', 'PETIT LARCENY', 'VEHICLE AND TRAFFIC LAWS', 'OFF. AGNST PUB ORD SENSBLTY &', 'INTOXICATED & IMPAIRED DRIVING', 'OFFENSES AGAINST PUBLIC ADMINI', 'CRIMINAL TRESPASS', 'OTHER STATE LAWS', 'OFFENSES AGAINST THE PERSON', 'MISCELLANEOUS PENAL LAW', 'GRAND LARCENY OF MOTOR VEHICLE', 'FORGERY', 'UNAUTHORIZED USE OF A VEHICLE', 'DANGEROUS DRUGS', 'OTHER STATE LAWS (NON PENAL LAW)', 'THEFT-FRAUD', 'DISORDERLY CONDUCT', 'ADMINISTRATIVE CODE', 'RAPE', 'KIDNAPPING & RELATED OFFENSES', 'PROSTITUTION & RELATED OFFENSES', 'POSSESSION OF STOLEN PROPERTY', 'CHILD ABANDONMENT/NON SUPPORT 1', 'PETIT LARCENY OF MOTOR VEHICLE', 'DISRUPTION OF A RELIGIOUS SERV', 'CANNABIS RELATED OFFENSES', 'OTHER OFFENSES RELATED TO THEFT', 'JOSTLING', "BURGLAR'S TOOLS", 'ALCOHOLIC BEVERAGE CONTROL LAW', 'OFFENSES AGAINST PUBLIC SAFETY', 'ANTICIPATORY OFFENSES', 'GAMBLING', 'FRAUDULENT ACCOSTING', 'OFFENSES RELATED TO CHILDREN', '(null)', 'ESCAPE 3', 'INTOXICATED/IMPAIRED DRIVING', 'UNLAWFUL POSS. WEAP. ON SCHOOL', 'HOMICIDE-NEGLIGENT,UNCLASSIFIE', 'OTHER TRAFFIC INFRACTION', 'FORTUNE TELLING', 'LOITERING/GAMBLING (CARDS, DIC', 'LOITERING', 'NEW YORK CITY HEALTH CODE']
offences_dict = {
    "Murder": 7042,
    "Manslaughter": 1822,
    "Attempted murder": 1411,
    "Sexual assault - level 3": 1047,
    "Discharging firearm with intent": 988,
    "Sexual assault - level 2": 678,
    "Robbery": 583,
    "Assault - level 3": 405,
    "Using firearm in commission of an offence": 267,
    "Sexual assault - level 1": 211,
    "Breaking and entering": 187,
    "Luring a person under 18 via computer": 172,
    "Theft over $5,000": 139,
    "Fraud": 109,
    "Weapons possession": 88,
    "Theft of a motor vehicle": 84,
    "Assault - level 2": 77,
    "Counterfeiting currency": 69,
    "Uttering threat to person": 46,
    "Criminal harassment": 45,
    "Theft under $5,000": 37,
    "Mischief": 30,
    "Assault - level 1": 23,
    "Fail to appear": 16,
    "Disturb the peace": 9,
    "Cannabis possession": 7
}
dictionary_mapping = {
    "HARRASSMENT 2": "Criminal harassment",
    "FRAUDS": "Fraud",
    "DANGEROUS WEAPONS": "Weapons possession",
    "FELONY ASSAULT": "Assault - level 3",
    "ARSON": "Mischief",
    "ROBBERY": "Robbery",
    "BURGLARY": "Breaking and entering",
    "SEX CRIMES": "Sexual assault - level 2",
    "MURDER & NON-NEGL. MANSLAUGHTER": "Murder",
    "ASSAULT 3 & RELATED OFFENSES": "Assault - level 3",
    "GRAND LARCENY": "Theft over $5,000",
    "CRIMINAL MISCHIEF & RELATED OF": "Mischief",
    "OFFENSES INVOLVING FRAUD": "Fraud",
    "PETIT LARCENY": "Theft under $5,000",
    "VEHICLE AND TRAFFIC LAWS": "Theft of a motor vehicle",
    "OFF. AGNST PUB ORD SENSBLTY &": "Disturb the peace",
    "INTOXICATED & IMPAIRED DRIVING": "Cannabis possession",
    "OFFENSES AGAINST PUBLIC ADMINI": "Disturb the peace",
    "CRIMINAL TRESPASS": "Breaking and entering",
    "OTHER STATE LAWS": "Disturb the peace",
    "OFFENSES AGAINST THE PERSON": "Assault - level 2",
    "MISCELLANEOUS PENAL LAW": "Disturb the peace",
    "GRAND LARCENY OF MOTOR VEHICLE": "Theft of a motor vehicle",
    "FORGERY": "Counterfeiting currency",
    "UNAUTHORIZED USE OF A VEHICLE": "Theft of a motor vehicle",
    "DANGEROUS DRUGS": "Cannabis possession",
    "OTHER STATE LAWS (NON PENAL LAW)": "Disturb the peace",
    "THEFT-FRAUD": "Fraud",
    "DISORDERLY CONDUCT": "Disturb the peace",
    "ADMINISTRATIVE CODE": "Disturb the peace",
    "RAPE": "Sexual assault - level 3",
    "KIDNAPPING & RELATED OFFENSES": "Luring a person under 18 via computer",
    "PROSTITUTION & RELATED OFFENSES": "Sexual assault - level 1",
    "POSSESSION OF STOLEN PROPERTY": "Theft under $5,000",
    "CHILD ABANDONMENT/NON SUPPORT 1": "Luring a person under 18 via computer",
    "PETIT LARCENY OF MOTOR VEHICLE": "Theft of a motor vehicle",
    "DISRUPTION OF A RELIGIOUS SERV": "Mischief",
    "CANNABIS RELATED OFFENSES": "Cannabis possession",
    "OTHER OFFENSES RELATED TO THEFT": "Theft under $5,000",
    "JOSTLING": "Disturb the peace",
    "BURGLAR'S TOOLS": "Breaking and entering",
    "ALCOHOLIC BEVERAGE CONTROL LAW": "Disturb the peace",
    "OFFENSES AGAINST PUBLIC SAFETY": "Disturb the peace",
    "ANTICIPATORY OFFENSES": "Disturb the peace",
    "GAMBLING": "Disturb the peace",
    "FRAUDULENT ACCOSTING": "Fraud",
    "OFFENSES RELATED TO CHILDREN": "Luring a person under 18 via computer",
    "(null)": "Disturb the peace",
    "ESCAPE 3": "Fail to appear",
    "INTOXICATED/IMPAIRED DRIVING": "Cannabis possession",
    "UNLAWFUL POSS. WEAP. ON SCHOOL": "Weapons possession",
    "HOMICIDE-NEGLIGENT,UNCLASSIFIE": "Manslaughter",
    "OTHER TRAFFIC INFRACTION": "Disturb the peace",
    "FORTUNE TELLING": "Disturb the peace",
    "LOITERING/GAMBLING (CARDS, DIC": "Disturb the peace",
    "LOITERING": "Disturb the peace",
    "NEW YORK CITY HEALTH CODE": "Disturb the peace"
}
crimes_data['Offence'] = crimes_data['OFNS_DESC'].map(dictionary_mapping)
crimes_data["Offence weight"] = crimes_data['Offence'].map(offences_dict)

In [6]:
columns_to_keep = ["CMPLNT_FR_DT", "CRM_ATPT_CPTD_CD", "SUSP_AGE_GROUP", "SUSP_RACE", "SUSP_SEX", "Latitude", "Longitude", "Offence", "Offence weight"]
crimes_data = crimes_data[columns_to_keep]
columns_mapping = {
    "CMPLNT_FR_DT": "Date",
    "CRM_ATPT_CPTD_CD": "Status",
    "SUSP_AGE_GROUP": "Suspect age",
    "SUSP_RACE": "Suspect race",
    "SUSP_SEX": "Suspect sex",
    "Offence": "Offence",
    "Offence weight": "Offence weight"
}
crimes_data = crimes_data.rename(columns=columns_mapping)
crimes_data['Date'] = pd.to_datetime(crimes_data['Date'], errors='coerce', format='%m/%d/%Y')
#Keep only data in XXI century
crimes_data = crimes_data[crimes_data['Date'].dt.year >= 2000]
crimes_data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
crimes_data = crimes_data[["Latitude", "Longitude", "Offence", "Offence weight"]]
geometry = gpd.points_from_xy(crimes_data.Longitude, crimes_data.Latitude)
crimes_gdf = gpd.GeoDataFrame(crimes_data, geometry=geometry)
crimes_gdf.crs = "EPSG:4326"
crimes_gdf.reset_index(drop=True, inplace=True)
crimes_gdf.head()

,Latitude,Longitude,Offence,Offence weight,geometry
0,40.545217,-74.169591,Criminal harassment,45,POINT (-74.16959 40.54522)
1,40.582305,-74.169067,Criminal harassment,45,POINT (-74.16907 40.5823)
2,40.638927,-74.167396,Fraud,109,POINT (-74.1674 40.63893)
3,40.626851,-74.163438,Weapons possession,88,POINT (-74.16344 40.62685)
4,40.631331,-74.155867,Assault - level 3,405,POINT (-74.15587 40.63133)


In [7]:
table_name = "crimes"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    offence VARCHAR(255) NOT NULL,
    offence_weight INT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (latitude, longitude, offence, offence_weight, geometry) VALUES (%s, %s, %s, %s, %s)
"""
for i, row in tqdm(crimes_gdf.iterrows(), total=crimes_gdf.shape[0]):
    cursor.execute(insert_script, (row['Latitude'], row['Longitude'], row['Offence'], row['Offence weight'], str(row['geometry'])))
DB.commit()

100%|██████████| 433198/433198 [01:17<00:00, 5583.08it/s]


# Bus stops

In [8]:
bus_stops_data = pd.read_csv('Data/bus_stops.csv')
columns_to_keep = ["stop_lat", "stop_lon"]
bus_stops_data = bus_stops_data[columns_to_keep]
bus_stops_data = bus_stops_data.rename(columns={"stop_lat": "Latitude", "stop_lon": "Longitude"})
geometry = gpd.points_from_xy(bus_stops_data.Longitude, bus_stops_data.Latitude)
bus_stops_gdf = gpd.GeoDataFrame(bus_stops_data, geometry=geometry)
bus_stops_gdf.crs = "EPSG:4326"
bus_stops_gdf.reset_index(drop=True, inplace=True)
bus_stops_gdf.head()

,Latitude,Longitude,geometry
0,40.731342,-73.990292,POINT (-73.99029 40.73134)
1,40.732608,-73.989958,POINT (-73.98996 40.73261)
2,40.733936,-73.989720,POINT (-73.98972 40.73394)
3,40.738976,-73.987053,POINT (-73.98705 40.73898)
4,40.766274,-73.969329,POINT (-73.96933 40.76627)


In [9]:
table_name = "bus_stops"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (latitude, longitude, geometry) VALUES (%s, %s, %s)
"""
for i, row in tqdm(bus_stops_gdf.iterrows(), total=bus_stops_gdf.shape[0]):
    cursor.execute(insert_script, (row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()

100%|██████████| 15045/15045 [00:02<00:00, 5873.49it/s]


# Subway stations

In [10]:
subway_stations_data = pd.read_csv('Data/subway_stations.csv')
columns_to_keep = ["stop_lat", "stop_lon"]
subway_stations_data = subway_stations_data[columns_to_keep]
subway_stations_data = subway_stations_data.rename(columns={"stop_lat": "Latitude", "stop_lon": "Longitude"})
geometry = gpd.points_from_xy(subway_stations_data.Longitude, subway_stations_data.Latitude)
subway_stations_gdf = gpd.GeoDataFrame(subway_stations_data, geometry=geometry)
subway_stations_gdf.crs = "EPSG:4326"
subway_stations_gdf.reset_index(drop=True, inplace=True)
subway_stations_gdf.head()

,Latitude,Longitude,geometry
0,40.889248,-73.898583,POINT (-73.89858 40.88925)
1,40.884667,-73.900870,POINT (-73.90087 40.88467)
2,40.878856,-73.904834,POINT (-73.90483 40.87886)
3,40.903125,-73.850620,POINT (-73.85062 40.90312)
4,40.898379,-73.854376,POINT (-73.85438 40.89838)


In [11]:
table_name = "subway_stations"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (latitude, longitude, geometry) VALUES (%s, %s, %s)
"""
for i, row in tqdm(subway_stations_gdf.iterrows(), total=subway_stations_gdf.shape[0]):
    cursor.execute(insert_script, (row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()

100%|██████████| 493/493 [00:00<00:00, 5135.37it/s]


# Restaurants

In [12]:
restaurants_data = pd.read_csv('Data/restaurants.csv')
columns_to_keep = ["Latitude", "Longitude"]
restaurants_data = restaurants_data[columns_to_keep]
restaurants_data.dropna(subset=['Latitude', 'Longitude'], inplace=True)
restaurants_data.drop_duplicates(subset=["Latitude", "Longitude"], inplace=True, keep='first')
geometry = gpd.points_from_xy(restaurants_data.Longitude, restaurants_data.Latitude)
restaurants_gdf = gpd.GeoDataFrame(restaurants_data, geometry=geometry)
restaurants_gdf.crs = "EPSG:4326"
restaurants_gdf.reset_index(drop=True, inplace=True)
restaurants_gdf.head()

,Latitude,Longitude,geometry
0,40.625573,-74.024248,POINT (-74.02425 40.62557)
1,40.826960,-73.926410,POINT (-73.92641 40.82696)
2,40.751973,-73.939317,POINT (-73.93932 40.75197)
3,40.715505,-73.747712,POINT (-73.74771 40.71551)
4,40.717838,-73.989279,POINT (-73.98928 40.71784)


In [13]:
table_name = "restaurants"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (latitude, longitude, geometry) VALUES (%s, %s, %s)
"""
for i, row in tqdm(restaurants_gdf.iterrows(), total=restaurants_gdf.shape[0]):
    cursor.execute(insert_script, (row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()

100%|██████████| 24051/24051 [00:04<00:00, 5720.98it/s]


# Points of interest

In [14]:
poi_data = pd.read_csv('Data/Points_of_Interest.csv')
poi_data.head()

,the_geom,SEGMENTID,COMPLEXID,SAFTYPE,SOS,PLACEID,FACI_DOM,BIN,BOROUGH,CREATED,MODIFIED,FACILITY_T,SOURCE,B7SC,PRI_ADD,NAME
0,POINT (-74.00701717096757 40.724634757833414),31895,0,N,1.0,567,9,0,1.0,05/14/2009 12:00:00 AM,11/18/2011 12:00:00 AM,6,DoITT,19743001.0,0,HOLLAND
1,POINT (-73.82661642130311 40.797182526598505),306303,3378,N,2.0,568,8,0,4.0,05/14/2009 12:00:00 AM,01/09/2017 12:00:00 AM,6,DoITT,49731001.0,0,WHITESTONE
2,POINT (-73.99395441100663 40.70384707235758),144842,3960,N,2.0,576,8,0,3.0,05/14/2009 12:00:00 AM,01/22/2018 12:00:00 AM,6,DoITT,39734001.0,0,BROOKLYN
3,POINT (-73.9919414213091 40.70960010711745),162664,0,N,1.0,580,8,0,1.0,05/14/2009 12:00:00 AM,05/11/2011 12:00:00 AM,6,DoITT,19795001.0,0,MANHATTAN
4,POINT (-73.9526609766105 40.73906602249743),157362,0,N,1.0,582,8,0,3.0,05/14/2009 12:00:00 AM,03/03/2017 12:00:00 AM,6,DoITT,39740001.0,0,PULASKI


In [15]:
poi_data['Longitude'] = poi_data['the_geom'].apply(lambda x: float(x.split()[1][1:]))
poi_data['Latitude'] = poi_data['the_geom'].apply(lambda x: float(x.split()[2][:-1]))
columns_to_keep = ["NAME", "FACILITY_T", "Latitude", "Longitude"]
poi_data = poi_data[columns_to_keep]
poi_data.rename(columns={"NAME": "Name", "FACILITY_T": "Type"}, inplace=True)
geometry = gpd.points_from_xy(poi_data.Longitude, poi_data.Latitude)
poi_gdf = gpd.GeoDataFrame(poi_data, geometry=geometry)
poi_gdf.crs = "EPSG:4326"
poi_gdf.head()

,Name,Type,Latitude,Longitude,geometry
0,HOLLAND,6,40.724635,-74.007017,POINT (-74.00702 40.72463)
1,WHITESTONE,6,40.797183,-73.826616,POINT (-73.82662 40.79718)
2,BROOKLYN,6,40.703847,-73.993954,POINT (-73.99395 40.70385)
3,MANHATTAN,6,40.709600,-73.991941,POINT (-73.99194 40.7096)
4,PULASKI,6,40.739066,-73.952661,POINT (-73.95266 40.73907)


## Education

In [16]:
education_gdf = poi_gdf.loc[poi_gdf['Type'] == 2]
education_gdf.drop('Type', axis=1, inplace=True)
education_gdf.reset_index(drop=True, inplace=True)
education_gdf.head()

,Name,Latitude,Longitude,geometry
0,LONG ISLAND CITY HIGH SCHOOL,40.765515,-73.932713,POINT (-73.93271 40.76552)
1,PS/IS 268,40.707666,-73.785277,POINT (-73.78528 40.70767)
2,BENJAMIN N CARDOZO HIGH SCHOOL,40.751751,-73.756176,POINT (-73.75618 40.75175)
3,ACADEMY OF AMERICAN STUDIES HIGH SCHOOL,40.752275,-73.936447,POINT (-73.93645 40.75227)
4,JHS 185 EDWARD BLEEKER,40.775043,-73.818535,POINT (-73.81854 40.77504)


In [17]:
table_name = "education"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    name VARCHAR(255) NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (name, latitude, longitude, geometry) VALUES (%s, %s, %s, %s)
"""
for i, row in tqdm(education_gdf.iterrows(), total=education_gdf.shape[0]):
    cursor.execute(insert_script, (row['Name'], row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()

  0%|          | 0/3813 [00:00<?, ?it/s]

100%|██████████| 3813/3813 [00:00<00:00, 5857.12it/s]


## Culture

In [18]:
cultural_gdf = poi_gdf.loc[poi_gdf['Type'] == 3]
cultural_gdf=cultural_gdf.drop('Type', axis=1)
cultural_gdf.reset_index(drop=True, inplace=True)
cultural_gdf.head()

,Name,Latitude,Longitude,geometry
0,SQUADRON A ARMORY,40.785674,-73.953958,POINT (-73.95396 40.78567)
1,DIA ART FOUNDATION CHELSEA GALLERY,40.747846,-74.006168,POINT (-74.00617 40.74785)
2,AMERICAN MUSEUM OF NATURAL HISTORY,40.781065,-73.973560,POINT (-73.97356 40.78106)
3,BAYSIDE LIBRARY,40.760170,-73.768341,POINT (-73.76834 40.76017)
4,BROAD CHANNEL LIBRARY,40.600757,-73.820052,POINT (-73.82005 40.60076)


In [19]:
table_name = "cultural"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    name VARCHAR(255) NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (name, latitude, longitude, geometry) VALUES (%s, %s, %s, %s)
"""
for i, row in tqdm(cultural_gdf.iterrows(), total=cultural_gdf.shape[0]):
    cursor.execute(insert_script, (row['Name'], row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()

  0%|          | 0/590 [00:00<?, ?it/s]

100%|██████████| 590/590 [00:00<00:00, 5673.12it/s]


## Recreation

In [20]:
recreation_gdf = poi_gdf.loc[poi_gdf['Type'] == 4]
recreation_gdf = recreation_gdf.drop('Type', axis=1)
recreation_gdf.reset_index(drop=True, inplace=True)
recreation_gdf.head()

,Name,Latitude,Longitude,geometry
0,ST NICHOLAS PLAYGROUND,40.813787,-73.950852,POINT (-73.95085 40.81379)
1,VAN CORTLANDT PARK SOUTHWEST PLAYGROUND,40.886694,-73.898068,POINT (-73.89807 40.88669)
2,KOSSUTH PLAYGROUND,40.877823,-73.882623,POINT (-73.88262 40.87782)
3,MULLALY PLAYGROUND 1,40.831347,-73.925385,POINT (-73.92538 40.83135)
4,ST MARYS PLAYGROUND EAST,40.810316,-73.910446,POINT (-73.91045 40.81032)


In [21]:
table_name = "recreation"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    name VARCHAR(255) NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (name, latitude, longitude, geometry) VALUES (%s, %s, %s, %s)
"""
for i, row in tqdm(recreation_gdf.iterrows(), total=recreation_gdf.shape[0]):
    cursor.execute(insert_script, (row['Name'], row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()

100%|██████████| 4109/4109 [00:00<00:00, 5730.84it/s]


## Religion

In [22]:
religious_gdf = poi_gdf.loc[poi_gdf['Type'] == 9]
religious_gdf = religious_gdf.drop('Type', axis=1)
religious_gdf.reset_index(drop=True, inplace=True)
religious_gdf.head()

,Name,Latitude,Longitude,geometry
0,COLUMBIA UNIVERSITY ST PAULS CHAPEL,40.807804,-73.960883,POINT (-73.96088 40.8078)
1,AAA SFARD CONGREGATION,40.637924,-73.898000,POINT (-73.898 40.63792)
2,TIMES SQUARE CHURCH,40.762582,-73.984331,POINT (-73.98433 40.76258)
3,HOLY NATIVITY EPISCOPAL CHURCH,40.873590,-73.879797,POINT (-73.8798 40.87359)
4,FIRST PRESBYTERIAN CHURCH,40.734500,-73.995165,POINT (-73.99517 40.7345)


In [23]:
table_name = "religious"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    name VARCHAR(255) NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (name, latitude, longitude, geometry) VALUES (%s, %s, %s, %s)
"""
for i, row in tqdm(religious_gdf.iterrows(), total=religious_gdf.shape[0]):
    cursor.execute(insert_script, (row['Name'], row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()

  0%|          | 0/1594 [00:00<?, ?it/s]

100%|██████████| 1594/1594 [00:00<00:00, 5692.91it/s]


## Health

In [24]:
health_gdf = poi_gdf.loc[poi_gdf['Type'] == 10]
health_gdf = health_gdf.drop('Type', axis=1)
health_gdf.reset_index(drop=True, inplace=True)
health_gdf.head()

,Name,Latitude,Longitude,geometry
0,CONEY ISLAND HOSPITAL,40.586122,-73.965638,POINT (-73.96564 40.58612)
1,NYU LANGONE MEDICAL CENTER,40.742171,-73.974266,POINT (-73.97427 40.74217)
2,JACOBI HOSPITAL NURSES RESIDENCE,40.854608,-73.846554,POINT (-73.84655 40.85461)
3,NY BLOOD CENTER VERNON BLVD,40.747844,-73.952093,POINT (-73.95209 40.74784)
4,EINSTEIN COLLEGE CHANIN INSTITUTE FOR CANCER R...,40.850033,-73.845842,POINT (-73.84584 40.85003)


In [25]:
table_name = "health"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    name VARCHAR(255) NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (name, latitude, longitude, geometry) VALUES (%s, %s, %s, %s)
"""
for i, row in tqdm(health_gdf.iterrows(), total=health_gdf.shape[0]):
    cursor.execute(insert_script, (row['Name'], row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()

100%|██████████| 270/270 [00:00<00:00, 6136.21it/s]


# Main attractions

I will create a dataset with 15 main attrractions in NYC based on this site: https://www.tripadvisor.com/Attractions-g60763-Activities-New_York_City_New_York.html.

In [26]:
main_attractions_data = pd.DataFrame({
    "Name": ["Central Park", "The National 9/11 Memorial & Museum", "The Metropolitan Museum of Art", "Empire State Building", "The High Line", "Top of the Rock", "Statue of Liberty", "Times Square", "Broadway", "Grand Central Terminal"],
    "Latitude": [40.785091, 40.7115, 40.779437, 40.748817, 40.747993, 40.7591, 40.689247, 40.7580, 40.7590, 40.7527],
    "Longitude": [-73.968285, -74.0133, -73.963244, -73.985428, -74.004765, -73.9794, -74.044502, -73.9855, -73.9845, -73.9772]
})
geometry = gpd.points_from_xy(main_attractions_data.Longitude, main_attractions_data.Latitude)
main_attractions_gdf = gpd.GeoDataFrame(main_attractions_data, geometry=geometry)
main_attractions_gdf.crs = "EPSG:4326"
main_attractions_gdf.head()

,Name,Latitude,Longitude,geometry
0,Central Park,40.785091,-73.968285,POINT (-73.96828 40.78509)
1,The National 9/11 Memorial & Museum,40.711500,-74.013300,POINT (-74.0133 40.7115)
2,The Metropolitan Museum of Art,40.779437,-73.963244,POINT (-73.96324 40.77944)
3,Empire State Building,40.748817,-73.985428,POINT (-73.98543 40.74882)
4,The High Line,40.747993,-74.004765,POINT (-74.00476 40.74799)


In [27]:
table_name = "main_attractions"
drop_table_script = f"""
DROP TABLE IF EXISTS {table_name};
"""
cursor.execute(drop_table_script)
create_script = f"""
CREATE TABLE {table_name} (
    id INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
    name VARCHAR(255) NOT NULL,
    latitude FLOAT NOT NULL,
    longitude FLOAT NOT NULL,
    geometry TEXT NOT NULL
)
"""
cursor.execute(create_script)
insert_script = f"""
INSERT INTO {table_name} (name, latitude, longitude, geometry) VALUES (%s, %s, %s, %s)
"""
for i, row in tqdm(main_attractions_gdf.iterrows(), total=main_attractions_gdf.shape[0]):
    cursor.execute(insert_script, (row['Name'], row['Latitude'], row['Longitude'], str(row['geometry'])))
DB.commit()
cursor.close()

100%|██████████| 10/10 [00:00<00:00, 5000.36it/s]
